In [1]:
!pip install gradio
!pip install -U 'https://github.com/ms-anlaytics/Image-labeling-with-Zero-Shot'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.8 MB 7.6 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 84 kB 1.3 MB/s 
     |████████████████████████████████| 278 kB 65.4 MB/s 
     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 84 kB 2.8 MB/s 
     |████████████████████████████████| 2.3 MB 46.8 MB/s 
     |████████████████████████████████| 106 kB 57.1 MB/s 
     |████████████████████████████████| 64 kB 3.4 MB/s 
     |████████████████████████████████| 80 kB 10.7 MB/s 
     |████████████████████████████████| 68 kB 7.4 MB/s 
     |████████████████████████████████| 58 kB 7.2 MB/s 
     |████████████████████████████████| 50 kB 7.7 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4711 sha256=8cbe907d7e30c00ffb2c52b326cb7c5e6f5bcf7e4bdacbc4c11fea11288e2fd6
  Stored in direc

In [2]:
!pip install flatten-json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for flatten-json: filename=flatten_json-0.1.13-py3-none-any.whl size=7978 sha256=cbcd240f9eff59138aff61492f39d08edc299ac6f9004a703c4239a0f0eff405
  Stored in directory: /root/.cache/pip/wheels/87/c5/6d/7a772fecd8d6ebae9e60d997f74b9a96ead7d5a0f26a920090
Successfully built flatten-json


In [3]:
import glob 
import requests  
import gradio as gr
import pandas as pd
from flatten_json import flatten
from PIL import Image
from IPython.core.display import HTML
import os


In [4]:
def path_to_image_html(file):
    '''
      Changes the image file name into HTML tag
      
    '''
    return '<img src=\"'+file+'\" style=\"width:128px;height:128px;\" >'


In [5]:
def format_img_name(path,dir):
  '''
    Accepts path to the image folder
    And directory name of the image folder
    Shortens the file names
  '''
  os.chdir(dir)
  for img in glob.glob(path):
    file_name=img.replace('ISSUE','')
    os.rename(img, file_name)

In [6]:
def save_HTML(df, src):
  '''
    Saves data to HTML file
    Takes a dataframe and file list of image names
  '''
  shot_data_html = df.assign(image=src)
  shot_data_html.to_html(dir + '/webpage.html',escape=False)
  

In [7]:
def save_CSV(df, file_name):
  '''
    Saves data to csv file
    Takes a dataframe and file list of image names
  '''
  shot_data = df.assign(image=file_name)
  shot_data.to_csv('./shot.csv')
  

In [8]:
from sre_constants import error
def zero_shot(path, dir):
  shot_data=pd.DataFrame()
   
  src=[]
  file_name=[]

  images = glob.glob(os.path.join(dir, str(r'*.jpg')))
  for img in images:
    print(img)
    img_b64 = gr.processing_utils.encode_url_or_file_to_base64(img)
    try:
      r = requests.post(url='https://hf.space/embed/Datatrooper/zero-shot-image-classification/+/api/predict/', json={"data": [img_b64,"solar panel,other"]})
      result = r.json()
      
      shot_data=shot_data.append([flatten(d) for d in result['data']])

      print(r)
      new_name=img.replace(dir+'./','')
      
      src.append(path_to_image_html(new_name))
      file_name.append(new_name)
    except:
      print('failed')
      continue

  save_HTML(shot_data, src)
  save_CSV(shot_data, file_name)
    

 

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [10]:
%cd '/content/gdrive/My Drive/'
%mkdir Images

/content/gdrive/My Drive
mkdir: cannot create directory ‘Images’: File exists


In [11]:
%cd /content/gdrive/My Drive/Images

/content/gdrive/My Drive/Images


In [14]:
path = r"*.jpg"
dir='/content/gdrive/My Drive/Images/'
zero_shot(path, dir)



/content/gdrive/My Drive/Images/image (2).jpg
<Response [200]>
/content/gdrive/My Drive/Images/image (9).jpg
<Response [200]>
/content/gdrive/My Drive/Images/image (1).jpg
<Response [200]>
/content/gdrive/My Drive/Images/image (5).jpg
<Response [200]>
/content/gdrive/My Drive/Images/image (4).jpg
<Response [200]>
/content/gdrive/My Drive/Images/image (3).jpg
<Response [200]>
/content/gdrive/My Drive/Images/image (6).jpg
<Response [200]>
/content/gdrive/My Drive/Images/image (7).jpg
<Response [200]>
/content/gdrive/My Drive/Images/image (8).jpg
<Response [200]>


In [15]:
csvFile = pd.read_csv('shot.csv', index_col=0)
csvFile.head()

,label,confidences_0_label,confidences_0_confidence,confidences_1_label,confidences_1_confidence,image
0,solar panel,solar panel,0.999573,other,0.000427,/content/gdrive/My Drive/Images/image (2).jpg
0,solar panel,solar panel,0.999776,other,0.000224,/content/gdrive/My Drive/Images/image (9).jpg
0,solar panel,solar panel,0.999818,other,0.000182,/content/gdrive/My Drive/Images/image (1).jpg
0,solar panel,solar panel,0.998002,other,0.001998,/content/gdrive/My Drive/Images/image (5).jpg
0,solar panel,solar panel,0.999804,other,0.000196,/content/gdrive/My Drive/Images/image (4).jpg


In [16]:
target_images=[]

# % confidence for image classification
confidence=0.89


for index, row in csvFile.iterrows():
  if (row.confidences_0_confidence > confidence) and (row.confidences_0_label ==('solar panel')):
        target_images.append(row)
  else:
    continue

  

In [17]:
dir='/content/gdrive/My Drive/Images'
os.chdir(dir)
target_images=pd.DataFrame(target_images)
src=[]
for index, image in target_images.iterrows():
  src.append(path_to_image_html(image.image))
 
save_HTML(target_images, src)